# 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取手寫辨識資料集
wine = datasets.load_wine()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 0.146 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

0.14672894294872418


In [10]:
# 設定要訓練的超參數組合
n_estimators = [100, 150, 200, 250, 300]
max_depth = [1, 2, 3, 4, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=5)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] max_depth=1, n_estimators=100 ...................................
[CV] max_depth=1, n_estimators=100 ...................................
[CV] max_depth=1, n_estimators=100 ...................................
[CV] max_depth=1, n_estimators=150 ...................................
[CV]  max_depth=1, n_estimators=100, score=-0.09392992433679272, total=   0.0s
[CV]  max_depth=1, n_estimators=100, score=-0.05886671534980027, total=   0.0s
[CV]  max_depth=1, n_estimators=100, score=-0.09120879366069432, total=   0.0s
[CV] max_depth=1, n_estimators=150 ...................................
[CV] max_depth=1, n_estimators=150 ...................................
[CV] max_depth=1, n_estimators=200 ...................................
[CV]  max_depth=1, n_estimators=150, score=-0.06177615622655461, total=   0.0s
[CV] max_depth=1, n_estimators=200 ...................................
[CV]  max_depth=1, n_estimators=150, score=-0.087759279

[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.7s


[CV]  max_depth=4, n_estimators=100, score=-0.06866565289165169, total=   0.1s
[CV] max_depth=4, n_estimators=100 ...................................
[CV]  max_depth=4, n_estimators=200, score=-0.06866565289165169, total=   0.1s
[CV] max_depth=4, n_estimators=200 ...................................
[CV]  max_depth=4, n_estimators=100, score=-0.12081286368623095, total=   0.0s
[CV] max_depth=4, n_estimators=100 ...................................
[CV]  max_depth=3, n_estimators=200, score=-0.12081286368623095, total=   0.1s
[CV] max_depth=4, n_estimators=250 ...................................
[CV]  max_depth=3, n_estimators=250, score=-0.12081286368623095, total=   0.1s
[CV] max_depth=3, n_estimators=250 ...................................
[CV]  max_depth=4, n_estimators=100, score=-0.0998676874461367, total=   0.0s
[CV] max_depth=4, n_estimators=150 ...................................
[CV]  max_depth=4, n_estimators=200, score=-0.12081286368623095, total=   0.1s
[CV] max_depth=4, n_es

[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    1.9s finished


In [11]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.078503 using {'max_depth': 2, 'n_estimators': 300}


In [12]:
grid_result.best_params_

{'max_depth': 2, 'n_estimators': 300}

In [13]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [15]:
# 調整參數後約可降至 0.054 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

0.03323846038161252
